# Exploration of factors influencing students’ PISA scores in Thailand with Machine Learning approaches : Classification Models 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_val_score

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
# ML Models
#import lightgbm as lgb
#from lightgbm import LGBMRegressor 
import xgboost as xg 
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
# Import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, \
accuracy_score, plot_roc_curve, roc_auc_score, recall_score, \
precision_score, f1_score, classification_report
#interpretml 
from interpret import show
from interpret.data import Marginal
from interpret.glassbox import ExplainableBoostingClassifier
from interpret.perf import RegressionPerf
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())
# Feature Importance 
import shap

In [2]:
#Import dataset
train_thailand = pd.read_csv('datasets/train_pisa2018_thailand.csv')
test_thailand = pd.read_csv('datasets/test_pisa2018_thailand.csv')

In [3]:
train_thailand.head(2)

,CNTSTUID,GRADE,AGE,ISCEDL,ISCEDD,ISCEDO,PARED,IMMIG,REPEAT,MMINS,...,CLSIZE,CREACTIV,EDUSHORT,STAFFSHORT,STUBEHA,TEACHBEHA,SCMCEG,"Is_MISCED_5A,6","Is_FISCED_5A,6","Is_HISCED_5A,6"
0,76400082.0,0.0,15.75,ISCED level 3,A,General,12.0,Native,Did not repeat a grade,240.0,...,33,2,1.2478,0.1077,0.1795,-0.5205,-1.0542,0,0,0
1,76401146.0,-1.0,15.42,ISCED level 2,A,General,6.0,Native,Did not repeat a grade,200.0,...,33,3,-1.4212,-1.4551,-3.3785,-0.7624,-0.1868,0,0,0


In [4]:
test_thailand.head(2)

,CNTSTUID,GRADE,AGE,ISCEDL,ISCEDD,ISCEDO,PARED,IMMIG,REPEAT,MMINS,...,CLSIZE,CREACTIV,EDUSHORT,STAFFSHORT,STUBEHA,TEACHBEHA,SCMCEG,"Is_MISCED_5A,6","Is_FISCED_5A,6","Is_HISCED_5A,6"
0,76406643.0,0.0,15.42,ISCED level 3,A,General,16.0,Native,Did not repeat a grade,50.0,...,48,3,-0.1130,-0.2968,-0.0384,0.9229,-1.4469,1,1,1
1,76407714.0,0.0,15.67,ISCED level 3,A,General,12.0,Native,Did not repeat a grade,110.0,...,43,2,2.1432,-0.4314,0.2268,-2.0409,-0.1868,0,0,0


In [5]:
train_thailand.isnull().sum().sum(), test_thailand.isnull().sum().sum()

(0, 0)

In [6]:
train_thailand.set_index('CNTSTUID', inplace=True)
test_thailand.set_index('CNTSTUID', inplace=True)

In [7]:
train_thailand[['TOTALSCORE']].describe()

,TOTALSCORE
count,6775.000000
mean,431.946357
std,85.666985
min,210.007233
25%,367.618350
50%,419.663433
75%,489.412667
max,707.016300


In [ ]:
def assign_performance_level(data):
    if data <= train_thailand['TOTALSCORE'].mean():
        return '0' #lowperformance 
    else:
         return '1'  #highperformance 

In [ ]:
train_thailand['PISALEVEL'] = train_thailand['TOTALSCORE'].apply(assign_performance_level)
test_thailand['PISALEVEL'] = test_thailand['TOTALSCORE'].apply(assign_performance_level)

In [ ]:
train_thailand['PISALEVEL'].value_counts()

In [ ]:
train_thailand['PISALEVEL'].value_counts(normalize=True).mul(100).round(2)

In [ ]:
test_thailand['PISALEVEL'].value_counts()

In [ ]:
test_thailand['PISALEVEL'].value_counts(normalize=True).mul(100).round(2)

In [ ]:
X_train =train_thailand[['PARED', 'SMINS', 'CHANGE', 'ESCS', 'UNDREM', 'METASUM',
       'METASPAM', 'HOMEPOS', 'HEDRES', 'ICTRES', 'EMOSUPS', 'JOYREAD',
       'WORKMAST', 'GCSELFEFF', 'PERSPECT', 'RESPECT', 'AWACOM',
       'DISCRIM', 'INTICT', 'SENWT', 'STRATIO', 'TOTAT', 'PROAT5AB', 'PROAT5AM', 'PROAT6',
       'CREACTIV', 'EDUSHORT', 'STUBEHA', 'Is_MISCED_5A,6',
       'Is_FISCED_5A,6', 'ISCEDL', 'ISCEDD','ISCEDO', 'IMMIG', 'REPEAT', 'SCCHANGE', 'GENDER','Dept', 'Region','SCHLTYPE', 'GRADE']]
X_train = pd.get_dummies(columns=[ 'Is_MISCED_5A,6','Is_FISCED_5A,6', 'ISCEDL', 'ISCEDD','ISCEDO', 'IMMIG', 'REPEAT',
                                  'SCCHANGE', 'GENDER','Dept', 'Region','SCHLTYPE'], drop_first =True, data =X_train)
y_train = train_thailand['PISALEVEL']


X_test =test_thailand[['PARED', 'SMINS', 'CHANGE', 'ESCS', 'UNDREM', 'METASUM',
       'METASPAM', 'HOMEPOS', 'HEDRES', 'ICTRES', 'EMOSUPS', 'JOYREAD',
       'WORKMAST', 'GCSELFEFF', 'PERSPECT', 'RESPECT', 'AWACOM',
       'DISCRIM', 'INTICT', 'SENWT', 'STRATIO', 'TOTAT', 'PROAT5AB', 'PROAT5AM', 'PROAT6',
       'CREACTIV', 'EDUSHORT', 'STUBEHA', 'Is_MISCED_5A,6',
       'Is_FISCED_5A,6', 'ISCEDL', 'ISCEDD','ISCEDO', 'IMMIG', 'REPEAT', 'SCCHANGE', 'GENDER','Dept', 'Region','SCHLTYPE', 'GRADE']]
X_test = pd.get_dummies(columns=[ 'Is_MISCED_5A,6','Is_FISCED_5A,6', 'ISCEDL', 'ISCEDD','ISCEDO', 'IMMIG', 'REPEAT', 'SCCHANGE', 
                                 'GENDER','Dept', 'Region','SCHLTYPE'], drop_first =True, data =X_test)
y_test = test_thailand['PISALEVEL']

In [ ]:
scaler = StandardScaler()

X_train_sc = scaler.fit_transform(X_train)

X_test_sc = scaler.transform(X_test)

In [ ]:
from sklearn.dummy import DummyClassifier
baseline_model = DummyClassifier()
baseline_model.fit(X_train_sc, y_train)
baseline_preds = baseline_model.predict(X_test_sc)
ConfusionMatrixDisplay.from_predictions(y_test, baseline_preds);

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, baseline_preds))

In [ ]:
logreg =LogisticRegression()


parameters = [{'penalty':['l1','l2']}, 
              {'C':[1, 10, 100]}]

logreg_gr = GridSearchCV(estimator = logreg,  
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           verbose=0)

logreg_gr.fit(X_train_sc, y_train)


In [ ]:
preds= logreg_gr.predict(X_test_sc)

In [ ]:
pd.DataFrame(logreg_gr.cv_results_).sort_values('rank_test_score').head(5)[['params','mean_test_score']]

In [ ]:
cm = confusion_matrix(y_test, preds)
# visualize confusion matrix with seaborn heatmap

cm_matrix = pd.DataFrame(data=cm, columns=['Actual High:1', 'Actual Low:0'], 
                                 index=['Predict High:1', 'Predict Low:0'])

sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu');

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, preds))

In [ ]:
len(X_train.columns), len(logreg.coef_)

In [ ]:
logreg =LogisticRegression(random_state= 42, C= 10)
logreg.fit(X_train_sc, y_train)
preds= logreg.predict(X_test_sc)
pd.Series(logreg.coef_[0], index = X_train.columns).sort_values(ascending=False)

In [ ]:
ebm = ExplainableBoostingClassifier(random_state=42, feature_names =X_train.columns)
ebm.fit(X_train_sc, y_train)

In [ ]:
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)



In [ ]:
ebm.score(X_train_sc, y_train),ebm.score(X_test_sc, y_test)

In [ ]:
preds_ebm = ebm.predict(X_test_sc)

In [ ]:
cm_ebm = confusion_matrix(y_test, preds_ebm)
# visualize confusion matrix with seaborn heatmap

cm_matrix = pd.DataFrame(data=cm_ebm, columns=['Actual High:1', 'Actual Low:0'], 
                                 index=['Predict High:1', 'Predict Low:0'])

sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu');

In [ ]:
print(classification_report(y_test, preds_ebm))

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, random_state=42)

# fit the model to the training set

rfc.fit(X_train, y_train)

# Predict on the test set results

preds_rfc = rfc.predict(X_test)
print('Model accuracy score with 100 decision-trees : {0:0.4f}'. 
      format(accuracy_score(y_test, preds)))

In [ ]:
#Find important features with Random Forest model 

In [ ]:
# view the feature scores

feature_scores = pd.Series(rfc.feature_importances_, index=X_train.columns).sort_values(ascending=False)

feature_scores

In [ ]:
# Creating a seaborn bar plot

f, ax = plt.subplots(figsize=(30, 24))
ax = sns.barplot(x=feature_scores, y=feature_scores.index, data=train_thailand)
ax.set_title("Visualize feature scores of the features")
ax.set_yticklabels(feature_scores.index)
ax.set_xlabel("Feature importance score")
ax.set_ylabel("Features")
plt.show()

In [ ]:
cm_rfc = confusion_matrix(y_test, preds_rfc)
# visualize confusion matrix with seaborn heatmap

cm_matrix = pd.DataFrame(data=cm_rfc, columns=['Actual High:1', 'Actual Low:0'], 
                                 index=['Predict High:1', 'Predict Low:0'])

sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu');

In [ ]:
print(classification_report(y_test, preds_rfc))

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [ ]:
xgb = XGBClassifier(n_estimators = 200, learning_rate = 0.5, max_depth = 4)
xgb.fit(X_train_sc, y_train)

In [ ]:
preds_xgb = xgb.predict(X_test_sc)

In [ ]:
# view the feature scores

feature_scores = pd.Series(xgb.feature_importances_, index=X_train.columns).sort_values(ascending=False)

feature_scores

In [ ]:
cm_xgb = confusion_matrix(y_test, preds_xgb)
# visualize confusion matrix with seaborn heatmap

cm_matrix = pd.DataFrame(data=cm_xgb, columns=['Actual High:1', 'Actual Low:0'], 
                                 index=['Predict High:1', 'Predict Low:0'])

sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu');

In [ ]:
print(classification_report(y_test, preds_xgb))

In [ ]:
# explain the model's predictions using SHAP
explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(X_train_sc)

In [ ]:
shap_values = shap.TreeExplainer(xgb).shap_values(X_train_sc)
shap.summary_plot(shap_values, X_train_sc, plot_type="bar", feature_names= X_train.columns)

In [ ]:
xgb = XGBClassifier()
parameters = {'learning_rate': [.03, 0.05], #so called `eta` value
              'max_depth': [2, 3, 4],
              'n_estimators': [100, 200]}

xgb_grid = GridSearchCV(xgb,
                        parameters,
                        cv = 2,
                        verbose=True)

xgb_grid.fit(X_train,y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

In [ ]:
pd.DataFrame(xgb_grid.cv_results_).sort_values('rank_test_score').head(5)[['params','param_max_depth','mean_test_score']]